In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


from teste import diretorio_teste
project_path=diretorio_teste()

In [2]:
import os
import geopandas as gpd
import pandas as pd
from logging import (
    Logger,
    getLogger,
)
Logger=getLogger()

from notebooks.jupyter import utils
from utils import (
    save_parquet_excel,
    get_data_diretorio,
    check_crs
)

In [3]:
data_path = get_data_diretorio()

# Parques Urbanos e Lineares

# Dependências

Este notebook é dependente dos assets resultante dos notebooks '../../area_verde_publica/download_data/malha_parques_municipais.ipynb'

In [4]:
assets_path = os.path.join(
    data_path,
    'assets'
)

In [5]:
gdf_parq_mun = gpd.read_parquet(
    os.path.join(
        assets_path,
        'parques_municipais.parquet'
    )
)

# Conferir e Filtrar Parques Lineares e Urbanos

In [6]:
gdf_parq_mun.sample(2)

,cd_parq_mun,nm_parque,tx_tipo_ca,tx_tipo_si,cd_tipo_ca,cd_tipo_si,geometry
116,172,Horizonte Azul - Cardeal Rossi,Parque de Conservação,Proposto,2,1,"POLYGON ((318050.287 7373724.868, 318046.329 7..."
2,75,Mananciais Paulistanos - Ribeirão Borore,Parque de Conservação,Proposto,2,1,"POLYGON ((327212.474 7365887.385, 327338.995 7..."


O gdf_parq_mun tem todos os parques em São Paulo. E há vários que não nos interessam (ps.: para garantir que não tenha problemas por causa da caixa alta ou letra capital, vamos deixar tudo em caixa baixa, em inglês é chamada de lower):

In [7]:
gdf_parq_mun['tx_tipo_ca']=gdf_parq_mun['tx_tipo_ca'].str.lower()

In [8]:
uniques=gdf_parq_mun['tx_tipo_ca'].unique()
#.str.contains('inear', case=False)
uniques

array(['parque urbano', 'parque orla', 'parque de conservação',
       'parque natural', 'parque linear'], dtype=object)

O mais seguro é filtrar pelo cd do tipo de parque. No nosso caso, a coluna [cd_tipo_ca]. Para termos certeza de qual o cd de cada um dos parques, vamos filtrar primeiro procurando similaridade nos textos dos tipos de parques, e então encontrar o cd correspondente.

In [9]:
for u in uniques:
    print(u)
    if 'linear' in str(u):
        name_linear = u
    if 'urb' in str(u):
        name_urb=u
print (f'{name_linear} | {name_urb}')

parque urbano
parque orla
parque de conservação
parque natural
parque linear
parque linear | parque urbano


In [10]:
cd_linear=(
    gdf_parq_mun
    .loc[gdf_parq_mun['tx_tipo_ca']==name_linear, 'cd_tipo_ca']
    .unique()
)

print(cd_linear)

if int(cd_linear.shape[0])==1:
    cd_linear = int(cd_linear[0])
else:
    Logger.error(
        f'''Erro: o cd_tipo_ca ({
            cd_linear
        }) não corresponde completamente ao txt_tipo_ca({
            name_linear
        })'''
    )

cd_linear

[5]


5

In [11]:
cd_urb = (
    gdf_parq_mun
    .loc[gdf_parq_mun['tx_tipo_ca']==name_urb, 'cd_tipo_ca']
    .unique()
)

if int(cd_urb.shape[0])==1:
    cd_urb = int(cd_urb[0])
else:
    Logger.error(
        f'''Erro: o cd_tipo_ca ({
            cd_urb
        }) não corresponde completamente ao txt_tipo_ca({
            name_urb
        })'''
    )

cd_urb

1

Aqui descobrimos que o cd dos parques lineares (presente em cd_tipo_ca) é 5 e dos parques urbanos é 1. Vamos filtrar pelo cd, pois ele é mais confiável que o texto 

In [12]:
parques_line_urb=pd.concat([
    (
        gdf_parq_mun
        .loc[gdf_parq_mun['cd_tipo_ca']== cd_linear]
    ),
    (
        gdf_parq_mun
        .loc[gdf_parq_mun['cd_tipo_ca']==cd_urb]
    )
])

In [13]:
gdf_parq_mun.shape

(280, 7)

In [14]:
parques_line_urb.shape

(213, 7)

In [15]:
parques_line_urb.sample(3)

,cd_parq_mun,nm_parque,tx_tipo_ca,tx_tipo_si,cd_tipo_ca,cd_tipo_si,geometry
163,185,Vila Guilherme,parque urbano,Existente,1,2,"POLYGON ((336769.812 7398716.442, 336776.442 7..."
269,279,Parque do Bixiga,parque urbano,Proposto,1,1,"POLYGON ((332496.792 7394131.92, 332501.698 73..."
260,280,Banespa,parque urbano,Proposto,1,1,"POLYGON ((327582.895 7385463.753, 327578.876 7..."


# Conferir CRS

In [16]:
parques_line_urb= check_crs(parques_line_urb)

# Salvar GDF

In [17]:
save_parquet_excel(
    parques_line_urb,
    fname='parques_urbanos_e_lineares',
    data_path = data_path,
    data_subpath = 'assets'
)